## Training Cluster

In [32]:
!pipeline cluster switch aws_k8s_training

switched to context "aws_k8s_training".


### [Kubernetes View of Training Cluster](http://kubernetes.demo.pipeline.io/)

In [33]:
from IPython.display import display, HTML

html = '<iframe width=100% height=500px src="http://kubernetes.demo.pipeline.io">'
display(HTML(html))

## Scale Out Spark Worker

### [Spark Admin](http://spark.demo.pipeline.io)

In [5]:
!pipeline service scale spark 3

replicationcontroller "spark-worker-2-0-1" scaled


In [72]:
from IPython.display import display, HTML

html = '<iframe width=100% height=500px src="http://kubernetes.demo.pipeline.io">'
display(HTML(html))

## Generate Spark ML Decision Tree

### Setup SparkSession

In [1]:
from pyspark.sql import SparkSession

sparkSession = SparkSession.builder.getOrCreate()

### Load Training Dataset from S3 into Spark

In [2]:
data = sparkSession.read.format("csv") \
  .option("inferSchema", "true").option("header", "true") \
  .load("s3a://datapalooza/R/census.csv")

data.head()

Row(age=39, workclass='State-gov', education='Bachelors', education_num=13, marital_status='Never-married', occupation='Adm-clerical', relationship='Not-in-family', race='White', sex='Male', capital_gain=2174, capital_loss=0, hours_per_week=40, native_country='United-States', income='<=50K')

### Build Spark ML Pipeline with Decision Tree Classifier

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import DecisionTreeClassifier

formula = RFormula(formula = "income ~ .")
classifier = DecisionTreeClassifier()

pipeline = Pipeline(stages = [formula, classifier])

pipelineModel = pipeline.fit(data)

print(pipelineModel)

PipelineModel_4ce8b50df7dddb075889


In [4]:
print(pipelineModel.stages[1].toDebugString)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_42c49a6db1abc21cf096) of depth 5 with 57 nodes
  If (feature 23 in {0.0})
   If (feature 52 <= 7443.0)
    If (feature 22 <= 13.0)
     If (feature 54 <= 42.0)
      If (feature 0 <= 33.0)
       Predict: 0.0
      Else (feature 0 > 33.0)
       Predict: 0.0
     Else (feature 54 > 42.0)
      If (feature 9 in {0.0})
       Predict: 0.0
      Else (feature 9 not in {0.0})
       Predict: 0.0
    Else (feature 22 > 13.0)
     If (feature 54 <= 43.0)
      If (feature 0 <= 32.0)
       Predict: 0.0
      Else (feature 0 > 32.0)
       Predict: 0.0
     Else (feature 54 > 43.0)
      If (feature 0 <= 32.0)
       Predict: 0.0
      Else (feature 0 > 32.0)
       Predict: 0.0
   Else (feature 52 > 7443.0)
    If (feature 0 <= 20.0)
     If (feature 8 in {0.0})
      Predict: 0.0
     Else (feature 8 not in {0.0})
      Predict: 1.0
    Else (feature 0 > 20.0)
     If (feature 40 in {1.0})
      If (feature 0 <= 36.0)
       Predic

## Convert Spark ML Pipeline to PMML

In [5]:
from jpmml import toPMMLBytes

pmmlBytes = toPMMLBytes(sparkSession, data, pipelineModel)

print(pmmlBytes.decode("utf-8"))

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<PMML xmlns="http://www.dmg.org/PMML-4_3" version="4.3">
	<Header>
		<Application/>
		<Timestamp>2017-01-19T22:37:05Z</Timestamp>
	</Header>
	<DataDictionary>
		<DataField name="income" optype="categorical" dataType="string">
			<Value value="&lt;=50K"/>
			<Value value="&gt;50K"/>
		</DataField>
		<DataField name="workclass" optype="categorical" dataType="string">
			<Value value="Private"/>
			<Value value="Self-emp-not-inc"/>
			<Value value="Local-gov"/>
			<Value value="State-gov"/>
			<Value value="Self-emp-inc"/>
			<Value value="Federal-gov"/>
			<Value value="Without-pay"/>
		</DataField>
		<DataField name="education" optype="categorical" dataType="string">
			<Value value="HS-grad"/>
			<Value value="Some-college"/>
			<Value value="Bachelors"/>
			<Value value="Masters"/>
			<Value value="Assoc-voc"/>
			<Value value="11th"/>
			<Value value="Assoc-acdm"/>
			<Value value="10th"/>
			<Value value="7th-8th"/>
			<Value v

## Deployment Option 1:  Mutable Model Deployment

### AWS:  Deploy New Model to Live, Running Model Server

In [112]:
!pipeline cluster switch aws_k8s_predictions

switched to context "aws_k8s_predictions".


### [Kubernetes View of Prediction Cluster - AWS](http://kubernetes-aws.demo.pipeline.io/)

In [113]:
from IPython.display import display, HTML

html = '<iframe width=100% height=500px src="http://kubernetes-aws.demo.pipeline.io">'
display(HTML(html))

In [6]:
from urllib import request

update_url = 'http://prediction-pmml-aws.demo.pipeline.io/update-pmml/pmml_census'

update_headers = {}
update_headers['Content-type'] = 'application/xml'

req = request.Request(update_url, headers=update_headers, data=pmmlBytes)
resp = request.urlopen(req)

print(resp.status) # Should return Http Status 200 

200


### GCP:  Deploy New Model to Live, Running Model Server

In [115]:
!pipeline cluster switch gcp_k8s_predictions

switched to context "gcp_k8s_predictions".


### [Kubernetes View of Prediction Cluster - GCP](http://kubernetes-gcp.demo.pipeline.io/)

In [71]:
from IPython.display import display, HTML

html = '<iframe width=100% height=500px src="http://kubernetes-gcp.demo.pipeline.io">'
display(HTML(html))

In [7]:
from urllib import request

update_url = 'http://prediction-pmml-gcp.demo.pipeline.io/update-pmml/pmml_census'

update_headers = {}
update_headers['Content-type'] = 'application/xml'

req = request.Request(update_url, headers=update_headers, data=pmmlBytes)
resp = request.urlopen(req)

print(resp.status) # Should return Http Status 200 

200


## Predict on New Data

### AWS

In [8]:
from urllib import request

evaluate_url = 'http://prediction-pmml-aws.demo.pipeline.io/evaluate-pmml/pmml_census'

evaluate_headers = {}
evaluate_headers['Content-type'] = 'application/json'
input_params = '{"age":39,"workclass":"State-gov","education":"Bachelors","education_num":13,"marital_status":"Never-married","occupation":"Adm-clerical","relationship":"Not-in-family","race":"White","sex":"Male","capital_gain":2174,"capital_loss":0,"hours_per_week":40,"native_country":"United-States"}' 
encoded_input_params = input_params.encode('utf-8')

req = request.Request(evaluate_url, headers=evaluate_headers, data=encoded_input_params)
resp = request.urlopen(req)

print(resp.read()) # Should return valid classification with probabilities

b'{"results":[[{\'income\': \'NodeScoreDistribution{result=<=50K, probability_entries=[<=50K=0.9564524694636218, >50K=0.04354753053637812], entityId=7, confidence_entries=[]}\'}]]'


### GCP

In [9]:
from urllib import request
import json

evaluate_url = 'http://prediction-pmml-gcp.demo.pipeline.io/evaluate-pmml/pmml_census'

evaluate_headers = {}
evaluate_headers['Content-type'] = 'application/json'
input_params = '{"age":39,"workclass":"State-gov","education":"Bachelors","education_num":13,"marital_status":"Never-married","occupation":"Adm-clerical","relationship":"Not-in-family","race":"White","sex":"Male","capital_gain":2174,"capital_loss":0,"hours_per_week":40,"native_country":"United-States"}' 
encoded_input_params = input_params.encode('utf-8')

req = request.Request(evaluate_url, headers=evaluate_headers, data=encoded_input_params)
resp = request.urlopen(req)

print(resp.read()) # Should return valid classification with probabilities

b'{"results":[{"result": "fallback"}]'


## Load Test Predictions Across AWS and Google Cloud

In [59]:
!pipeline cluster switch aws_k8s_training

switched to context "aws_k8s_training".


In [60]:
!pipeline service deploy loadtest

...Starting Load Test...
replicationcontroller "loadtest" created


### [View of Prediction Services](http://hystrix.demo.pipeline.io/hystrix-dashboard/monitor/monitor.html?streams=%5B%7B%22name%22%3A%22Predictions%20-%20AWS%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine-aws.demo.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%2C%7B%22name%22%3A%22Predictions%20-%20GCP%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine-gcp.demo.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%5D)

In [14]:
from IPython.display import display, HTML

html = '<iframe width=100% height=500px src="http://hystrix.demo.pipeline.io/hystrix-dashboard/monitor/monitor.html?streams=%5B%7B%22name%22%3A%22Predictions%20-%20AWS%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine-aws.demo.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%2C%7B%22name%22%3A%22Predictions%20-%20GCP%22%2C%22stream%22%3A%22http%3A%2F%2Fturbine-gcp.demo.pipeline.io%2Fturbine.stream%22%2C%22auth%22%3A%22%22%2C%22delay%22%3A%22%22%7D%5D">'
display(HTML(html))

## Scale Out Load Test

In [70]:
!pipeline cluster switch aws_k8s_training

switched to context "aws_k8s_training".


In [68]:
!pipeline service scale loadtest 2

replicationcontroller "loadtest" scaled


## Scale Out Model Prediction Servers

### AWS

In [51]:
!pipeline cluster switch aws_k8s_predictions

switched to context "aws_k8s_predictions".


In [78]:
!pipeline service scale prediction 2

replicationcontroller "prediction-pmml" scaled
replicationcontroller "prediction-codegen" scaled
Error from server: replicationcontrollers "prediction-cache" not found
replicationcontroller "prediction-tensorflow" scaled


### GCP

In [49]:
!pipeline cluster switch gcp_k8s_predictions

switched to context "gcp_k8s_predictions".


In [81]:
!pipeline service scale prediction 2

replicationcontroller "prediction-pmml" scaled
replicationcontroller "prediction-codegen" scaled
Error from server: replicationcontrollers "prediction-cache" not found
replicationcontroller "prediction-tensorflow" scaled


## Deployment Option 2:  Immutable Model Deployment

### Save Model to Disk

In [13]:
with open('/root/pipeline/prediction.ml/pmml/data/pmml_census/pmml_census.pmml', 'wb') as f:
  f.write(pmmlBytes)

!cat /root/pipeline/prediction.ml/pmml/data/pmml_census/pmml_census.pmml

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<PMML xmlns="http://www.dmg.org/PMML-4_3" version="4.3">
	<Header>
		<Application/>
		<Timestamp>2017-01-19T22:37:05Z</Timestamp>
	</Header>
	<DataDictionary>
		<DataField name="income" optype="categorical" dataType="string">
			<Value value="&lt;=50K"/>
			<Value value="&gt;50K"/>
		</DataField>
		<DataField name="workclass" optype="categorical" dataType="string">
			<Value value="Private"/>
			<Value value="Self-emp-not-inc"/>
			<Value value="Local-gov"/>
			<Value value="State-gov"/>
			<Value value="Self-emp-inc"/>
			<Value value="Federal-gov"/>
			<Value value="Without-pay"/>
		</DataField>
		<DataField name="education" optype="categorical" dataType="string">
			<Value value="HS-grad"/>
			<Value value="Some-college"/>
			<Value value="Bachelors"/>
			<Value value="Masters"/>
			<Value value="Assoc-voc"/>
			<Value value="11th"/>
			<Value value="Assoc-acdm"/>
			<Value value="10th"/>
			<Value value="7th-8th"/>
			<Value v

### Commit to Github and Trigger Canary Model Deployment

### Monitor Canary Model Deployment

In [45]:
from IPython.display import display, HTML

html = '<iframe width=100% height=500px src="http://airflow.demo.pipeline.io">'
display(HTML(html))

## Scale In and Cleanup

### Load Test

In [47]:
!pipeline cluster switch aws_k8s_training

switched to context "aws_k8s_training".


In [57]:
!pipeline service undeploy loadtest

...Loadtest...
...Ignore Any Errors...
replicationcontroller "loadtest" deleted


### Spark

In [20]:
!pipeline cluster switch aws_k8s_training

switched to context "aws_k8s_training".


In [21]:
!pipeline service scale spark 1

replicationcontroller "spark-worker-2-0-1" scaled


### AWS Predictions

In [54]:
!pipeline cluster switch aws_k8s_predictions

switched to context "aws_k8s_predictions".


In [23]:
!pipeline service scale prediction 1

replicationcontroller "prediction-pmml" scaled
replicationcontroller "prediction-codegen" scaled


### GCP Predictions

In [56]:
!pipeline cluster switch gcp_k8s_predictions

switched to context "gcp_k8s_predictions".


In [26]:
!pipeline service scale prediction 1

replicationcontroller "prediction-pmml" scaled
replicationcontroller "prediction-codegen" scaled
